In [8]:
import numpy as np
import cv2

with open('yolo\coco.names') as f:
    labels = [line.strip() for line in f]
model = cv2.dnn.readNet('yolo\yolov3.weights', 'yolo\cfg\yolov3.cfg')
layers_names_all = model.getLayerNames()
layers_names_output = [layers_names_all[i - 1] 
                       for i in model.getUnconnectedOutLayers()]

def countVehicles(video):    
    probability_minimum = 0.5
    threshold = 0.3
    ret, frame = video.read()
    if not ret:
        return
    h, w = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True)
    model.setInput(blob)
    output_from_network = model.forward(layers_names_output)

    bounding_boxes = []
    confidences = []
    class_numbers = []
    classes = {1:'bicycle', 2:'car', 3:'motorbike', 5:'bus', 7:'truck' }
    
    for result in output_from_network:
        for detected_objects in result:

            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]
            
            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
  
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    count = 0
    for i in results.flatten():
        if class_numbers[i] in classes.keys():
            count += 1
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            width, height = bounding_boxes[i][2], bounding_boxes[i][3]               

            cv2.rectangle(frame, (x_min, y_min), (x_min + width, y_min + height),
                          (0,255,0), 2)

            cv2.putText(frame, str(count), (x_min, y_min-5), 0, 0.5, (0,255,0), 2)
    frame = cv2.resize(frame, (1200, 650))
    cv2.imshow("window", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return count

cam = []
cam.append(cv2.VideoCapture('videos//1.mp4'))
cam.append(cv2.VideoCapture('videos//2.mp4'))
cam.append(cv2.VideoCapture('videos//3.mp4'))
cam.append(cv2.VideoCapture('videos//5.mp4'))

totaltime = 0
for i in range(4):
    vehicles = countVehicles(cam[i])
    basetime = 10
    maxtime = 40
    slot = 5
    time = (vehicles//slot)*5
    time += basetime
    if time > maxtime:
        time = maxtime
    totaltime += time
    print("Vehicles detected in lane",i+1,vehicles,"and time given", time)

for video in cam:
    video.release()

Vehicles detected in lane 1 10 and time given 20
Vehicles detected in lane 2 22 and time given 30
Vehicles detected in lane 3 7 and time given 15
Vehicles detected in lane 4 19 and time given 25
